# Boston House Pricing Prediction using Machine Learning

Dibuat oleh: Anthony Kevin Oktavius

Library Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Dataset Preparation

In [ ]:
housing_df = pd.read_csv("boston_housing.csv")
housing_df

## Data Visualization

## Data Preprocessing

## Modelling